In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.io as pio
pio.renderers.default = "iframe"

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

#Function Graph

In [2]:
#Define the function for graph

def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()
    from IPython.display import display, HTML
    fig_html = fig.to_html()
    display(HTML(fig_html))

#TSLA Data

##Data from Yahoo Finance

In [3]:
TSLA = yf.Ticker('TSLA')

TSLA_data = TSLA.history(period="max")
TSLA_data.reset_index(inplace=True)
TSLA_data.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


##Data from Webscrapping

In [4]:
URL =  'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm'

TSLA_revenue = requests.get(URL).text

In [5]:
soup = BeautifulSoup(TSLA_revenue, 'html.parser')
#soup

In [6]:
TSLA_revenue_data = pd.DataFrame(columns = ['Date','Revenue'])
#TSLA_revenue_data

In [7]:
for row in soup.find_all("tbody")[1].find_all('tr'):
    col = row.find_all("td")
    date = col[0].text
    Revenue = col[1].text

    # Finally we append the data of each row to the table
    TSLA_revenue_data = pd.concat([TSLA_revenue_data, pd.DataFrame({'Date':[date],"Revenue":[Revenue]})], ignore_index=True)

In [8]:
tesla_revenue =  TSLA_revenue_data

tesla_revenue['Revenue'] = tesla_revenue['Revenue'].str.replace(',|\$',"", regex=True)
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]
tesla_revenue.head(5)

,Date,Revenue
0,2022-09-30,21454
1,2022-06-30,16934
2,2022-03-31,18756
3,2021-12-31,17719
4,2021-09-30,13757


##TSLA GRAPH

In [9]:
make_graph(TSLA_data, tesla_revenue, 'TSLA')

Output hidden; open in https://colab.research.google.com to view.

#GME Data

##Data from Yahoo Finance

In [10]:
GME = yf.Ticker('GME')

GME_data = GME.history(period="max")
GME_data.reset_index(inplace=True)
GME_data.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693349,1.603295,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712708,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0


##Data from Webscrapping

In [11]:
URL =  'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'

GME_revenue = requests.get(URL).text
soup = BeautifulSoup(GME_revenue, 'html.parser')
GME_revenue_data = pd.DataFrame(columns = ['Date','Revenue'])

for row in soup.find_all("tbody")[1].find_all('tr'):
    col = row.find_all("td")
    date = col[0].text
    Revenue = col[1].text

    # Finally we append the data of each row to the table
    GME_revenue_data = pd.concat([GME_revenue_data, pd.DataFrame({'Date':[date],"Revenue":[Revenue]})], ignore_index=True)


GME_revenue =  GME_revenue_data

GME_revenue['Revenue'] = GME_revenue['Revenue'].str.replace(',|\$',"", regex=True)
GME_revenue.dropna(inplace=True)

GME_revenue = GME_revenue[GME_revenue['Revenue'] != ""]
GME_revenue.head(5)

,Date,Revenue
0,2020-04-30,1021
1,2020-01-31,2194
2,2019-10-31,1439
3,2019-07-31,1286
4,2019-04-30,1548


##GME GRAPH

In [12]:
make_graph(GME_data, GME_revenue, 'GME')

Output hidden; open in https://colab.research.google.com to view.